In [1]:
import tensorflow as tf
from tensorflow.keras import activations, layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.efficientnet import EfficientNetB7

In [2]:
def swish_function(x):
    return x * activations.sigmoid(x)

In [3]:
def squeeze_and_excitation_block(inputs, reduction=4):
    filters = inputs.shape[-1]
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1, 1, filters))(x)
    x = layers.Conv2D(filters // reduction, (1, 1), activation="relu", padding="same")(x)
    x = layers.Conv2D(filters, (1, 1), activation="sigmoid", padding="same")(x)
    return layers.multiply([inputs, x])

In [5]:
def mobile_inverted_bottleneck_conv_block(inputs, filters, kernel_size, strides, expand_ratio, reduction=4, dropout_rate=0.2):
    input_filters = inputs.shape[-1]
    expanded_filters = input_filters * expand_ratio

    if expand_ratio != 1:
        x = layers.Conv2D(expanded_filters, (1, 1), padding='same', use_bias=False)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(swish_function)(x)
    else:
        x = inputs

    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish_function)(x)

    x = squeeze_and_excitation_block(x, reduction=reduction)

    x = layers.Conv2D(filters, (1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if strides == 1 and input_filters == filters:
        if dropout_rate > 0:
            x = layers.Dropout(dropout_rate)(x)
        x = layers.add([x, inputs])
    
    return x

In [6]:
def build_efficientnet_b7(input_shape=(600, 600, 3), num_classes=1000):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(64, (3, 3), strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish_function)(x)
    
    x = mobile_inverted_bottleneck_conv_block(x, filters=32, kernel_size=3, strides=1, expand_ratio=1)
    x = mobile_inverted_bottleneck_conv_block(x, filters=32, kernel_size=3, strides=2, expand_ratio=6)
    
    x = mobile_inverted_bottleneck_conv_block(x, filters=64, kernel_size=3, strides=2, expand_ratio=6)
    x = mobile_inverted_bottleneck_conv_block(x, filters=64, kernel_size=3, strides=1, expand_ratio=6)

    x = mobile_inverted_bottleneck_conv_block(x, filters=128, kernel_size=5, strides=2, expand_ratio=6)
    x = mobile_inverted_bottleneck_conv_block(x, filters=128, kernel_size=5, strides=1, expand_ratio=6)

    x = mobile_inverted_bottleneck_conv_block(x, filters=256, kernel_size=5, strides=2, expand_ratio=6)
    x = mobile_inverted_bottleneck_conv_block(x, filters=256, kernel_size=5, strides=1, expand_ratio=6)

    x = mobile_inverted_bottleneck_conv_block(x, filters=512, kernel_size=3, strides=1, expand_ratio=6)
    x = mobile_inverted_bottleneck_conv_block(x, filters=512, kernel_size=3, strides=1, expand_ratio=6)

    x = layers.Conv2D(2048, (1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish_function)(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    
    return model

In [ ]:
efficientnet_b7_scratch_model = build_efficientnet_b7()
efficientnet_b7_scratch_model.summary()

In [ ]:
efficientnet_b7_tf_model = EfficientNetB7(include_top=True, weights="imagenet", input_shape=(600, 600, 3))
efficientnet_b7_tf_model.summary()